### Deep Structured Semantic Model - Prediction Evaluation
This notebook is used to evaluate the prediction performance of the DSSM model. It is different from the real time recommendation. 

#### Pre-requisites
- The model is trained and the index is created in the notebooks `DSSM Model.ipynb` and `DSSM Index (Faiss).ipynb`.
- All required files are saved in the `Saved_Triplet_Hinge_Loss` folder.

In [22]:
from general_program import *
import warnings
warnings.filterwarnings("ignore")

In [ ]:
save_folder_path='Saved_Triplet_Hinge_Loss/'

user_model, item_model, user_id_encoder, business_id_encoder, categories_encoder, user_scaler, business_scaler = load_saved_models(save_folder_path)

In [24]:
user_df, business_df, review_df, user_continuous_features_scaled, business_continuous_features_scaled, num_users, num_businesses, num_categories = prepare_data(user_df, business_df, review_df, categories_df, user_id_encoder, business_id_encoder, categories_encoder, user_scaler, business_scaler, use_stage='test')

# check if business_category_map
business_category_map = business_df.set_index('business_id_encoded')['category_encoded']

In [25]:
business_review_count = review_df.groupby('business_id').size()
business_avg_review = review_df.groupby('business_id')['stars'].mean()
business_df = business_df.merge(business_review_count.rename('review_count'), on='business_id', how='left').fillna({'review_count': 0})
business_df = business_df.merge(business_avg_review.rename('avg_review'), on='business_id', how='left').fillna({'avg_review': 0})

In [26]:
# Split review_df into train and test sets
train_data, test_data = train_test_split(review_df, test_size=0.2, random_state=42)

# Split the test set into positive and negative samples
positive_reviews = test_data[test_data['stars'] >= 4]
negative_reviews = test_data[test_data['stars'] < 4]

print(f"Number of positive reviews: {len(positive_reviews)}")
print(f"Number of negative reviews: {len(negative_reviews)}")
print(f"Total number of reviews: {len(test_data)}")
print(f"Ratio of positive to negative reviews: {len(positive_reviews) / len(negative_reviews):.2f}")

Number of positive reviews: 135929
Number of negative reviews: 60155
Total number of reviews: 196084
Ratio of positive to negative reviews: 2.26


In [27]:
def balance_test_data(positive_reviews, negative_reviews):
    # down-sample the positive reviews to balance the dataset
    positive_reviews_downsampled = positive_reviews.sample(n=len(negative_reviews), random_state=42)

    # combine the down-sampled positive reviews with the negative reviews
    balanced_test_data = pd.concat([positive_reviews_downsampled, negative_reviews], ignore_index=True)

    # shuffle the balanced test data
    balanced_test_data = balanced_test_data.sample(frac=1, random_state=42).reset_index(drop=True)

    # new statistics for the balanced test data
    positive_reviews = balanced_test_data[balanced_test_data['stars'] >= 4]
    negative_reviews = balanced_test_data[balanced_test_data['stars'] <= 2]

    print(f"Number of positive reviews: {len(positive_reviews)}")
    print(f"Number of negative reviews: {len(negative_reviews)}")
    print(f"Total number of reviews: {len(balanced_test_data)}")
    print(f"Ratio of positive to negative reviews: {len(positive_reviews) / len(negative_reviews):.2f}")
    return balanced_test_data

In [28]:
# balance the test data, comment this line to use the original test data
# test_data = balance_test_data(positive_reviews, negative_reviews)

# group the test data by user_id and get the business_id
test_data_grouped = test_data.groupby('user_id')['business_id'].apply(list).reset_index()

In [29]:
test_data['user_id_encoded'] = user_id_encoder.transform(test_data['user_id'])

# Step 2: Prepare inputs for user and business embeddings
# Extract user features
test_user_ids = test_data['user_id_encoded'].values
test_user_cont_features = user_scaler.transform(user_continuous_features_scaled.loc[test_user_ids].values)

# Extract business features
test_business_ids = test_data['business_id_encoded'].values
test_business_cont_features = business_scaler.transform(business_continuous_features_scaled.loc[test_business_ids].values)
test_business_categories = business_category_map.loc[test_business_ids].apply(
    lambda x: x if isinstance(x, list) else []
)
test_business_category_padded = pad_sequences(test_business_categories.tolist(), maxlen=5, padding="post")

In [30]:

# Step 3: Predict embeddings using the loaded models
test_user_embeddings = user_model.predict([test_user_ids, test_user_cont_features])
test_business_embeddings = item_model.predict([test_business_ids, test_business_category_padded, test_business_cont_features])

# Step 4: Compute cosine similarity for each user-business pair
def compute_cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Create a list of cosine similarities for each record in test_data
test_data['predicted_similarity'] = [
    compute_cosine_similarity(test_user_embeddings[i], test_business_embeddings[i])
    for i in range(len(test_data))
]

# Step 5: Set the predicted_label based on similarity score
test_data['predicted_label'] = (test_data['predicted_similarity'] >= 0).astype(int)

6128/6128 ━━━━━━━━━━━━━━━━━━━━ 6s 923us/step
6128/6128 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step  


In [31]:
# Extract ground truth labels and predicted scores
y_true = test_data['label'].values
y_pred_scores = test_data['predicted_similarity'].values  # Cosine similarity scores

# Convert similarity scores into binary predictions (threshold = 0 for cosine similarity)
y_pred_labels = (y_pred_scores >= 0).astype(int)

# Compute confusion matrix elements
true_positive = np.sum((y_true == 1) & (y_pred_labels == 1))
true_negative = np.sum((y_true == 0) & (y_pred_labels == 0))
false_positive = np.sum((y_true == 0) & (y_pred_labels == 1))
false_negative = np.sum((y_true == 1) & (y_pred_labels == 0))

# Compute dataset statistics
total_positive = np.sum(y_true)
total = len(y_true)
total_negative = total - total_positive

# Compute evaluation metrics
accuracy = (true_positive + true_negative) / total if total > 0 else 0
precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
recall = true_positive / total_positive if total_positive > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

# Weighted Fβ-score
beta = 2
f_beta = (1 + beta**2) * precision * recall / (beta**2 * precision + recall) if (beta**2 * precision + recall) > 0 else 0

# Compute dataset statistics
background_stats = pd.DataFrame({
    'Total Positive': [total_positive],
    'Total Negative': [total_negative],
    'Total': [total],
    'Ratio': [total_positive / total if total > 0 else 0],
})

print("Testing Data Statistics")
display(background_stats)

# Evaluation Metrics
evaluation_metric = pd.DataFrame({
    'Accuracy': [accuracy],
    'Precision': [precision],
    'Recall': [recall],
    'F1 Score': [f1_score],
    'F-beta Score': [f_beta],
    # 'Mean Reciprocal Rank': [mean_reciprocal_rank],
}).apply(lambda x: round(x, 4))

print("Evaluation Metrics")
display(evaluation_metric)

# Confusion Matrix
confusion_matrix = pd.DataFrame({
    'True Positive': [true_positive],
    'True Negative': [true_negative],
    'False Positive': [false_positive],
    'False Negative': [false_negative]
})

print("Confusion Matrix")
display(confusion_matrix)


Testing Data Statistics


,Total Positive,Total Negative,Total,Ratio
0,135929,60155,196084,0.693218


Evaluation Metrics


,Accuracy,Precision,Recall,F1 Score,F-beta Score
0,0.6037,0.7023,0.7434,0.7223,0.7348


Confusion Matrix


,True Positive,True Negative,False Positive,False Negative
0,101056,17315,42840,34873
